# Introduction to p8 python client for agentic systems
- Percolate is a database that allows agents to be built entirely in the data tier
- However either for testing or to orchastrate more complex agents in code, we provide a python guide
- This giude assumes that you have followed the guides to add some initial data and tools
- to learn more see here

In [ ]:

import sys
sys.path.append('../')
import requests
import percolate as p8

# Agents are simple in Percolate
- They are just Pydantic objects with references to extenral tools in your function repository
- You can provide structure and you can use this structure to save and search data in the database for your agent


In [ ]:
from pydantic import BaseModel,Field
import typing
#the default embedding field just sets json_schema_extra.embedding_provider so you can do that yourself
from percolate.models import DefaultEmbeddingField

class MyFirstAgent(BaseModel):
    """You are an agent that provides the information you are asked and a second random fact"""
    
    #because this has no config it will save to the `public` database schema
    
    name: str = Field(description="Task name")
    description:str = DefaultEmbeddingField(description="Task description")
    
    @classmethod
    def get_functions(cls):
        """i return a list of functions by key stored in the database"""
        return {}
    
#inlines functions are possible but we prefer declarative agents in Percolate
#use these to test but the recommended thing to do is add Python microservices or use databases directly via percolate
#you could ask p8.Agent(MyFirstAgent).run("when is sirshs birthday")
#     @classmethod
#     def some_helpful_function(cls, name:str):
#         """This function will tell you a peresons birthday
#         Its an example function that has typing and Args defined for tool use.
        
#         Args:
#             name: the persons names    
#         """
#         return "first of jan 2000"



In [ ]:
"""real world knowledge"""
p8.Agent(MyFirstAgent).run("what is the capital of ireland")

In [ ]:
# 4.test inline functions even though we dont use them
# 5.test restoring a function from the database as above
# 6.test planning over entities / functions
# 7.test using native functions
# 9.test loading entities and their functions
#10.test crud

## Behind the scenes
- we created a simple rest api wrapper instead of adding various api python clients
- you can can iterate over these check the response of each model for which you have an API key defined
- you should first check the langauge model is in your data store - either you have added the API token there or in the env var given by the reference to the env
- we will then iterate and ask the same question first with tools and then without tools to see what the payloads look like for testing
- **This is illustrated in `percolate.services.llm._check_all`**
- the high level wrapper we use parsers to a single response scheme

In [ ]:
from percolate.services.llm import _check_all, LanguageModel
#some example response are in the tests

### asking without tools

In [ ]:
gem = LanguageModel('gemini-1.5-flash')
gem.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

In [ ]:
claude = LanguageModel('claude-3-5-sonnet-20241022')
claude.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

In [ ]:
xai = LanguageModel('grok-2-latest')
xai.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

In [ ]:
deepseek = LanguageModel('deepseek-chat')
deepseek.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

### asking with tools
- we create a fictitious tool first - this tested the response in a simple way

In [ ]:
from percolate.services.llm import _check_all, LanguageModel

fns =[{
  "name": "get_weather",
  "description": "Get the weather forecast for a specific city and date",
  "parameters": {
    "type": "object",
    "properties": {
      "city": {
        "type": "string",
        "description": "The city for which to get the weather forecast"
      },
      "date": {
        "type": "string",
        "description": "The date for the weather forecast (YYYY-MM-DD)"
      }
    },
    "required": ["city", "date"]
  }
}]



In [ ]:
claude = LanguageModel('claude-3-5-sonnet-20241022')
claude.ask("What is the weather in dublin tomorrow", functions=None)

In [ ]:
gem = LanguageModel('gemini-1.5-flash')
gem.ask("What is the weather in dublin", functions=fns)

In [ ]:
gmin = LanguageModel('gpt-4o-mini')
gmin.ask("What is the weather in dublin tomorrow", functions=fns)

## An agent is just a pydantic class
- the docstring is the system prompt
- the fields are used as structured response if you ask for that
- the external functions are database or API tools you can use. 
- You can add functions on the agent directly to test but Percolate is designed for declarative agents that do not have inline functions

In [5]:
import percolate as p8
from pydantic import BaseModel,Field
import typing
from percolate.models import DefaultEmbeddingField
import uuid

class MyFirstAgent(BaseModel):
    """You are an agent that provides the information you are asked and a second random fact"""
    #because it has no config it will save to the public database schema
    
    id: uuid.UUID
    name: str = Field(description="Task name")
    #the default embedding field just settgs json_schema_extra.embedding_provider so you can do that yourself
    description:str = DefaultEmbeddingField(description="Task description")
    
    @classmethod
    def get_model_functions(cls):
        """i return a list of functions by key stored in the database"""
        return {
            'get_pet_findByStatus': "a function i use to look up pets based on their status"
        }
p8.repository(MyFirstAgent).register()

In [ ]:
from percolate.services import PostgresService
pg = PostgresService()
"""check the prompt that is created in the database"""
from IPython.display import Markdown
"""this is in the database so agents can be used in the database directly"""
Markdown(pg.execute(""" select * from p8.generate_markdown_prompt('MyFirstAgent') """)[0]['generate_markdown_prompt'])

In [ ]:
### althogh this structure is simple and we have not saved data we can take note of how the semantic information is used to generate queries
pg.execute(""" select * from p8.nl2sql('how would you get the task desc', 'MyFirstAgent') """)

### you can use the agent directly in python but just to observe its registered for use in the database we show that too

In [ ]:
from percolate.services import PostgresService
pg = PostgresService()
"""this query will take a few seconds WHEN we use the eval option (increase turns limit) because it 
1.loads the agent and 
2. calls the langauge model with prompt and available functions from the agent
3. the llm responds with a tool call and we make it returning a chunk of data 
4. and then it calls the api from the tool call and the response (this is the end of turn 1 and returns quickly as below)
5. turn 2 would then re-summarize the data using the langauge model again
but its a nice fast "tool call" if we dont increase turns!
"""
pg.execute(f""" select * from percolate_with_agent('list some pets that were sold', 'MyFirstAgent'); """)

### You can use lots of different models if you have the keys

In [ ]:
agent = p8.Agent(MyFirstAgent,allow_help=False)
agent.run("list two sold pets",language_model='grok-2-latest')
#agent.run("list two sold pets",language_model='deepseek-chat')
#agent.run("list two sold pets",language_model='groq-llama-3.3-70b-versatile')
#agent.run("list two sold pets",language_model='gemini-1.5-flash')
#agent.run("list two sold pets",language_model='claude-3-5-sonnet-20241022')
#agent.run("list two sold pets",language_model='cerebras-llama3.1-8b')

# we register APIs as follows
- Percolate feeds on data of two types; agents and tools
- Below is a open and free demo API that we can add as an example
- we can register the api - if we had a bearer token we could add that
- we can filter by verbs and endoints

In [ ]:
from percolate.utils.ingestion import add 
add.add_api('swagger_test', 'https://petstore.swagger.io/v2/swagger.json', verbs='get')

### we can check the function is there

In [ ]:
import percolate as p8
from percolate.models.p8 import Function

fn = p8.repository(Function).get_by_name(['get_pet_findByStatus'], as_model=True)
#this is a callable function that also provides the function description for langage models
fn